In [1]:
#Bibliotecas p/ importação 
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.metrics import confusion_matrix

sc = StandardScaler()# Método de Estandardização por Escalonamento

Using TensorFlow backend.


In [2]:
#Leitura dos dados de entrada X e dos rótulos de estabilidade y
X = pd.read_csv('C:\\Users\\Pablo\\Desktop\\tfc\\Simulações\\Data.csv', index_col = 0)
y = pd.read_csv('C:\\Users\\Pablo\\Desktop\\tfc\\Simulações\\y.csv', usecols = [1])

#Separação das variáveis - 86 variáveis ao todo
Dmax = X.iloc[:,:7]
Dmin = X.iloc[:,7:14]
D = X.iloc[:,:14]
PGmean = X.iloc[:,14:22]
PGmax = X.iloc[:,22:30]
PGmin = X.iloc[:,30:38]
PG = X.iloc[:,14:38]
QGmean = X.iloc[:,38:46]
QGmax = X.iloc[:,46:54]
QGmin = X.iloc[:,54:62]
QG = X.iloc[:,38:62]
VMmean = X.iloc[:,62:70]
VMmax = X.iloc[:,70:78]
VMmin = X.iloc[:,78:86]
VM = X.iloc[:,62:86]
    
parametros = {'Kernel':'linear', #Parâmetro p/ SVM - Tipo de kernel
             'Funcao perda':'binary_crossentropy',#Parâmetro p/ RNA - Função de pontuação de otimização
             'Otimizador':'SGD',#Parâmetro p/ RNA - Forma de cálculo de descida do gradiente do treinamento
             'Ativacao camadas':'relu',#Parâmetro p/ RNA - Função de ativação das camadas de entrada e escondidas
             'Ativacao saida':'sigmoid',#Parâmetro p/ RNA - Função de ativação da camada de saída
             }

conjuntos = {'D':D, 'PG':PG, 'QG':QG, 'VM':VM, 'Dmin':Dmin, 'Dmax':Dmax, 
             'PGmin':PGmin, 'PGmax':PGmax, 'QGmin':QGmin, 'QGmax':QGmax, 
             'VMmin':VMmin, 'VMmax':VMmax} #Dicionário com cada conjunto associado a um índice
    
index = list()
for x in conjuntos: index.append(x) #Lista de índices com os conjuntos

In [3]:
#Função para Treinamento e Teste de uma SVM
#Ao dicionário 'conjuntos' atribui-se os 12 conjuntos de dados definidos 
#A divisão dos dados é sempre aleatoria, portanto para cada execução da função é esperado resultados diferentes
#As saídas da função são as pontuações de precisão, uma de treino e outra de teste, obtidos na execução

def svm(conjuntos, y, C, kernel):
    precisao_treino = list()
    precisao_teste = list()
    for conjunto in conjuntos: #Para cada índice do dicionário 'conjuntos',
        X = conjuntos[str(conjunto)] # a variável X recebe o índice
        X_treino, X_teste, y_treino, y_teste = train_test_split(sc.fit_transform(abs(X.values)), y.values.ravel(),
                                                            test_size=0.3, shuffle = True) #Separação dos dados
        clf_svm = SVC(C=C, kernel=kernel) #É de interesse apenas C e kernel, com o resto dos parâmetros em default
        clf_svm.fit(X_treino, y_treino) #O Classificador se ajusta aos dados de treinamento
        precisao_treino.append(clf_svm.score(X_treino, y_treino)) #Pontuação de precisão de treino
        precisao_teste.append(clf_svm.score(X_teste, y_teste)) #Pontuação de precisão de teste
    return precisao_treino, precisao_teste

In [4]:
#Função p/ treinamento e teste de RNA 
#caso 0 - Apenas camadas de entrada e saída (2 layers)
#caso 1 - Uma única camada entre camadas de entrada e saída (3 layers)
#caso 2 - Quatro camadas entre camadas de entrada e saída (6 layers)
#A saída da função é uma única tabela resumo dos resultados obtidos na execução
def rna(conjuntos, index, caso, parametros):
    resultado = pd.DataFrame()
    perda = parametros['Funcao perda']
    otimizador = parametros['Otimizador']
    fatv_camadas = parametros['Ativacao camadas']
    fatv_camada_saida = parametros['Ativacao saida']
    
    for conjunto in conjuntos: #Para cada índice do dicionário 'conjuntos',
        X = conjuntos[str(conjunto)] # a variável X recebe o índice
        X_treino, X_teste, y_treino, y_teste = train_test_split(sc.fit_transform(abs(X.values)), y.values.ravel(),
                                                            test_size=0.3, shuffle = True) #Separação dos dados
        #Modelagem de camadas de neurônios
        modelo = Sequential()
        
        #Camada de Entrada  -> dimensão da entrada = número de variáveis
        modelo.add(Dense(64, input_dim=X_treino.shape[1], activation=fatv_camadas))
        
        if caso == 1:
            modelo.add(Dense(32, input_dim=64, activation=fatv_camadas))
        elif caso == 2:
            modelo.add(Dense(32, input_dim=64, activation=fatv_camadas))
            modelo.add(Dense(16, input_dim=32, activation=fatv_camadas))
            modelo.add(Dense(8, input_dim=16, activation=fatv_camadas))
            modelo.add(Dense(4, input_dim=8, activation=fatv_camadas))
            
        #Camada de Saída
        modelo.add(Dense(1, activation=fatv_camada_saida))
        
        #Compila o modelo - baseia-se no dicionário 'parametros'
        modelo.compile(loss=perda, 
                       optimizer=otimizador,
                       metrics=['accuracy'])
        
        #17.5% dos dados de treino p/ CV - 25% dos dados do conjunto
        #Batch size - número de amostras processadas antes de atualizar o modelo da RNA
        batch_size = 128
        
        #Steps per epoch - número de iterações de batch antes de terminar uma época = nº amostras/tamanho do batch
        steps_per_epoch = int(X_treino.shape[0]/batch_size)
        
        #Ajuste do modelo
        clf_rna = modelo.fit(X_treino,y_treino, validation_split=0.175, batch_size=None, epochs=30, 
                             validation_steps = steps_per_epoch*2, steps_per_epoch=steps_per_epoch)
        #Aavaliação do Modelo
        eval_treino=modelo.evaluate(X_treino, y_treino)
        eval_teste=modelo.evaluate(X_teste, y_teste)
        
        #Predição com os dados de teste
        y_pred = modelo.predict(X_teste)
        
        #Classificação p/ construção de matriz de confusão
        #y_pred>0.5 -> True -> Classificado corretamente; y_pred>0.5 -> False -> Classificado incorretamente
        y_pred = (y_pred>0.5)    
        cm = confusion_matrix(y_teste, y_pred)
        
        #Construção da tabela de saída
        resultado = resultado.append({'Teste - Acertos p/ Estável':cm[0][0],
                         'Teste - Erros p/ Estável':cm[0][1],
                         'Teste - Acertos p/ Instável':cm[1][1],
                         'Teste - Erros p/ Instável':cm[1][0],
                         'Precisão de Treino (%)':eval_treino[1]*100,
                         'Precisão de Teste (%)':eval_teste[1]*100}, ignore_index=True)
    resultado.set_index(np.array(index), inplace=True)
        
    return resultado

In [5]:
#Define-se a construção das saídas do programa

#A função 'saida_rna()' tem como saída 3 tabelas, executando a função os três casos de número de camadas de neurônios
def saida_rna(parametros, X, y):
    rna_2camadas = rna(conjuntos, index, 0, parametros)
    
    rna_3camadas = rna(conjuntos, index, 1, parametros)
    
    rna_6camadas = rna(conjuntos, index, 2, parametros)
    
    return rna_2camadas, rna_3camadas, rna_6camadas
#===============================================================================================================================
#A função 'saida_svm()' tem como saída duas tabelas constrúidas a partir da função 'svm()' para quatro valores de C
def saida_svm(parametros, X, y):
    svm_resultado_teste = pd.DataFrame({'Precisão de Teste (%), C=1': svm(conjuntos, y, 1, parametros['Kernel'])[1],
                                        'Precisão de Teste (%), C=0.1': svm(conjuntos, y, 0.1, parametros['Kernel'])[1],
                                        'Precisão de Teste (%), C=0.01': svm(conjuntos, y, 0.01, parametros['Kernel'])[1],
                                        'Precisão de Teste (%), C=0.001': svm(conjuntos, y, 0.001, parametros['Kernel'])[1]},
                                       index = index) #Construção da tabela de resultados da SVM para teste

    svm_resultado_treino = pd.DataFrame({'Precisão de Treino (%), C=1': svm(conjuntos, y, 1, parametros['Kernel'])[0],
                                     'Precisão de Treino (%), C=0.1': svm(conjuntos, y, 0.1, parametros['Kernel'])[0],
                                     'Precisão de Treino (%), C=0.01': svm(conjuntos, y, 0.01, parametros['Kernel'])[0],
                                     'Precisão de Treino (%), C=0.001': svm(conjuntos, y, 0.001, parametros['Kernel'])[0]},
                                        index = index) #Construção da tabela de resultados da SVM para treino
    #p/ resultado percentual
    svm_resultado_treino = svm_resultado_treino.apply(lambda x: x*100)
    svm_resultado_teste = svm_resultado_teste.apply(lambda x: x*100)
    
    return svm_resultado_treino, svm_resultado_teste

In [6]:
Treino_SVM = saida_svm(parametros, X, y)[0]
Teste_SVM = saida_svm(parametros, X, y)[1]

In [7]:
Resultado_RNA = saida_rna(parametros, X, y)

W0815 17:08:34.143594  9092 deprecation_wrapper.py:119] From C:\Users\Pablo\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 17:08:34.163480  9092 deprecation_wrapper.py:119] From C:\Users\Pablo\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 17:08:34.166472  9092 deprecation_wrapper.py:119] From C:\Users\Pablo\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0815 17:08:34.202412  9092 deprecation_wrapper.py:119] From C:\Users\Pablo\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Train on 2171 samples, validate on 461 samples
Epoch 1/30
20/20 [==============================] - 0s 11ms/step - loss: 0.9869 - acc: 0.1419 - val_loss: 0.8103 - val_acc: 0.2711
Epoch 2/30
20/20 [==============================] - 0s 3ms/step - loss: 0.6947 - acc: 0.5970 - val_loss: 0.6005 - val_acc: 0.9479
Epoch 3/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5435 - acc: 0.9684 - val_loss: 0.4879 - val_acc: 0.9740
Epoch 4/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4540 - acc: 0.9770 - val_loss: 0.4154 - val_acc: 0.9740
Epoch 5/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3918 - acc: 0.9779 - val_loss: 0.3621 - val_acc: 0.9740
Epoch 6/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3444 - acc: 0.9787 - val_loss: 0.3201 - val_acc: 0.9740
Epoch 7/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3063 - acc: 0.9809 - val_loss: 0.2858 - val_acc: 0.9783
Epoch 8/30
20/20 [========================

1128/1128 [==============================] - 0s 18us/step
Train on 2171 samples, validate on 461 samples
Epoch 1/30
20/20 [==============================] - 0s 12ms/step - loss: 0.7968 - acc: 0.3025 - val_loss: 0.6698 - val_acc: 0.4577
Epoch 2/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5993 - acc: 0.7135 - val_loss: 0.5293 - val_acc: 0.8243
Epoch 3/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4919 - acc: 0.8323 - val_loss: 0.4489 - val_acc: 0.8156
Epoch 4/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4268 - acc: 0.8412 - val_loss: 0.3975 - val_acc: 0.8460
Epoch 5/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3831 - acc: 0.8651 - val_loss: 0.3615 - val_acc: 0.8590
Epoch 6/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3514 - acc: 0.8780 - val_loss: 0.3344 - val_acc: 0.8720
Epoch 7/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3271 - acc: 0.8883 - val_loss: 0.3129 - 

1128/1128 [==============================] - 0s 19us/step
Train on 2171 samples, validate on 461 samples
Epoch 1/30
20/20 [==============================] - 0s 15ms/step - loss: 0.6232 - acc: 0.6938 - val_loss: 0.5626 - val_acc: 0.7332
Epoch 2/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5279 - acc: 0.7313 - val_loss: 0.4866 - val_acc: 0.7527
Epoch 3/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4629 - acc: 0.7645 - val_loss: 0.4310 - val_acc: 0.7983
Epoch 4/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4133 - acc: 0.8626 - val_loss: 0.3867 - val_acc: 0.9306
Epoch 5/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3732 - acc: 0.9604 - val_loss: 0.3505 - val_acc: 0.9696
Epoch 6/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3398 - acc: 0.9659 - val_loss: 0.3198 - val_acc: 0.9696
Epoch 7/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3112 - acc: 0.9659 - val_loss: 0.2932 - 

1128/1128 [==============================] - 0s 19us/step
Train on 2171 samples, validate on 461 samples
Epoch 1/30
20/20 [==============================] - 0s 17ms/step - loss: 0.6124 - acc: 0.7426 - val_loss: 0.5824 - val_acc: 0.7918
Epoch 2/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5504 - acc: 0.8384 - val_loss: 0.5280 - val_acc: 0.8850
Epoch 3/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5018 - acc: 0.8919 - val_loss: 0.4848 - val_acc: 0.9002
Epoch 4/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4628 - acc: 0.9127 - val_loss: 0.4494 - val_acc: 0.9241
Epoch 5/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4306 - acc: 0.9239 - val_loss: 0.4199 - val_acc: 0.9197
Epoch 6/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4036 - acc: 0.9259 - val_loss: 0.3950 - val_acc: 0.9219
Epoch 7/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3806 - acc: 0.9258 - val_loss: 0.3736 - 

...

1128/1128 [==============================] - 0s 26us/step
Train on 2171 samples, validate on 461 samples
Epoch 1/30
20/20 [==============================] - 1s 55ms/step - loss: 0.6213 - acc: 0.4979 - val_loss: 0.5992 - val_acc: 0.4707
Epoch 2/30
20/20 [==============================] - 0s 6ms/step - loss: 0.5639 - acc: 0.8215 - val_loss: 0.5602 - val_acc: 0.9913
Epoch 3/30
20/20 [==============================] - 0s 5ms/step - loss: 0.5322 - acc: 0.9931 - val_loss: 0.5359 - val_acc: 0.9913
Epoch 4/30
20/20 [==============================] - 0s 5ms/step - loss: 0.5078 - acc: 0.9931 - val_loss: 0.5115 - val_acc: 0.9913
Epoch 5/30
20/20 [==============================] - 0s 5ms/step - loss: 0.4800 - acc: 0.9931 - val_loss: 0.4828 - val_acc: 0.9913
Epoch 6/30
20/20 [==============================] - 0s 5ms/step - loss: 0.4519 - acc: 0.9931 - val_loss: 0.4552 - val_acc: 0.9913
Epoch 7/30
20/20 [==============================] - 0s 5ms/step - loss: 0.4247 - acc: 0.9931 - val_loss: 0.4284 - 

1128/1128 [==============================] - 0s 31us/step
Train on 2171 samples, validate on 461 samples
Epoch 1/30
20/20 [==============================] - 1s 59ms/step - loss: 0.6663 - acc: 0.4901 - val_loss: 0.6552 - val_acc: 0.5163
Epoch 2/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6551 - acc: 0.4937 - val_loss: 0.6413 - val_acc: 0.5293
Epoch 3/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6406 - acc: 0.5723 - val_loss: 0.6241 - val_acc: 0.6182
Epoch 4/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6229 - acc: 0.6615 - val_loss: 0.6033 - val_acc: 0.7419
Epoch 5/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6018 - acc: 0.7618 - val_loss: 0.5788 - val_acc: 0.8178
Epoch 6/30
20/20 [==============================] - 0s 6ms/step - loss: 0.5770 - acc: 0.8144 - val_loss: 0.5509 - val_acc: 0.8351
Epoch 7/30
20/20 [==============================] - 0s 6ms/step - loss: 0.5493 - acc: 0.8216 - val_loss: 0.5205 - 

1128/1128 [==============================] - 0s 29us/step
Train on 2171 samples, validate on 461 samples
Epoch 1/30
20/20 [==============================] - 1s 63ms/step - loss: 0.6878 - acc: 0.4942 - val_loss: 0.6854 - val_acc: 0.5293
Epoch 2/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6861 - acc: 0.4881 - val_loss: 0.6838 - val_acc: 0.5163
Epoch 3/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6845 - acc: 0.4819 - val_loss: 0.6823 - val_acc: 0.5184
Epoch 4/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6828 - acc: 0.4843 - val_loss: 0.6803 - val_acc: 0.5228
Epoch 5/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6810 - acc: 0.5208 - val_loss: 0.6788 - val_acc: 0.7115
Epoch 6/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6795 - acc: 0.7676 - val_loss: 0.6773 - val_acc: 0.7766
Epoch 7/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6780 - acc: 0.6956 - val_loss: 0.6757 - 

1128/1128 [==============================] - 0s 30us/step
Train on 2171 samples, validate on 461 samples
Epoch 1/30
20/20 [==============================] - 1s 67ms/step - loss: 0.6834 - acc: 0.6324 - val_loss: 0.6459 - val_acc: 0.7636
Epoch 2/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6293 - acc: 0.7473 - val_loss: 0.6060 - val_acc: 0.7896
Epoch 3/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6008 - acc: 0.7831 - val_loss: 0.5776 - val_acc: 0.8286
Epoch 4/30
20/20 [==============================] - 0s 6ms/step - loss: 0.5731 - acc: 0.8073 - val_loss: 0.5482 - val_acc: 0.8330
Epoch 5/30
20/20 [==============================] - 0s 6ms/step - loss: 0.5458 - acc: 0.8251 - val_loss: 0.5197 - val_acc: 0.8416
Epoch 6/30
20/20 [==============================] - 0s 7ms/step - loss: 0.5193 - acc: 0.8454 - val_loss: 0.4937 - val_acc: 0.8547
Epoch 7/30
20/20 [==============================] - 0s 6ms/step - loss: 0.4954 - acc: 0.8576 - val_loss: 0.4703 - 

1128/1128 [==============================] - 0s 31us/step


In [8]:
Treino_SVM

,"Precisão de Treino (%), C=1","Precisão de Treino (%), C=0.1","Precisão de Treino (%), C=0.01","Precisão de Treino (%), C=0.001"
D,100.000000,100.000000,100.000000,100.000000
PG,100.000000,100.000000,100.000000,96.352584
QG,99.962006,99.924012,99.772036,95.630699
VM,100.000000,100.000000,100.000000,98.822188
Dmin,99.316109,99.316109,98.252280,97.986322
Dmax,87.841945,83.738602,82.332827,81.534954
PGmin,93.427052,94.224924,93.085106,92.439210
PGmax,89.323708,83.130699,75.835866,62.879939
QGmin,78.723404,67.781155,62.689970,58.206687
QGmax,83.890578,83.776596,82.560790,77.697568


In [16]:
Resultado_RNA_2layers = Resultado_RNA[0]
Resultado_RNA_3layers = Resultado_RNA[1]
Resultado_RNA_6layers = Resultado_RNA[2]

In [17]:
Resultado_RNA_2layers

,Precisão de Teste (%),Precisão de Treino (%),Teste - Acertos p/ Estável,Teste - Acertos p/ Instável,Teste - Erros p/ Estável,Teste - Erros p/ Instável
D,100.000000,100.000000,542.0,586.0,0.0,0.0
PG,98.492908,98.632219,556.0,555.0,0.0,17.0
QG,96.631206,96.124620,564.0,526.0,3.0,35.0
VM,98.670213,98.252280,568.0,545.0,0.0,15.0
Dmin,97.783688,97.720365,555.0,548.0,0.0,25.0
Dmax,82.358156,82.674772,472.0,457.0,93.0,106.0
PGmin,92.996454,93.768997,533.0,516.0,35.0,44.0
PGmax,69.680851,70.364742,489.0,297.0,54.0,288.0
QGmin,59.840426,63.943769,536.0,139.0,0.0,453.0
QGmax,85.017730,82.940729,475.0,484.0,98.0,71.0
